# Exercise 1: Are ensembles better than their members?**Módulo:** Aprendizaje Automático y Aprendizaje Profundo**Alumna:** María Luisa Ros Bolea

## 0. Subida de datos a ColabEjecuto esta celda, se abre un cuadro para seleccionar archivos. Selecciono TODOS los .train y .test a la vez (los 26 archivos) y se suben a la carpeta correcta.

In [ ]:
import os
from google.colab import files

os.makedirs('exercise_1_data', exist_ok=True)

print("Selecciona todos los archivos .train y .test (los 26):")
uploaded = files.upload()

for filename in uploaded.keys():
    if filename.endswith('.train') or filename.endswith('.test'):
        dest = os.path.join('exercise_1_data', filename)
        with open(dest, 'wb') as f:
            f.write(uploaded[filename])

archivos = os.listdir('exercise_1_data')
n = len(archivos)
print("")
print("Archivos en exercise_1_data: " + str(n))
print("Todo listo.")

## 1. Fijar la semilla aleatoriaLo primero: fijo la semilla. Muchos algoritmos de ML tienen componentes aleatorios (inicialización de pesos, muestreo, particiones internas...) y sin semilla fija cada ejecución daría resultados distintos. Con `np.random.seed(42)` me aseguro de que todo sea reproducible. Que bastante caos tengo ya en mi vida como para añadir aleatoriedad a los experimentos.

In [ ]:
import numpy as np
np.random.seed(42)
print("Semilla fijada a 42. Reproducibilidad asegurada.")

Antes de nada, configuro las librerías y defino una paleta de colores pastel/rosa que voy a reutilizar en todos los gráficos del notebook. Así me ahorro repetir código y de paso queda todo coherente visualmente.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.metrics import accuracy_score
from sklearn.base import clone
warnings.filterwarnings('ignore')

# Configuro el estilo visual para TODO el notebook
plt.rcParams['figure.figsize'] = (14, 7)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.facecolor'] = '#FFF5F8'     # fondo rosa clarito
plt.rcParams['figure.facecolor'] = '#FFFFFF'
plt.rcParams['grid.alpha'] = 0.3
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.color'] = '#D5A6BD'

# Paleta rosa/pastel que reutilizo en todo
ROSA4 = ['#F48FB1', '#CE93D8', '#90CAF9', '#A5D6A7']  # 4 base learners
ROSA8 = ['#F48FB1', '#CE93D8', '#90CAF9', '#A5D6A7',  # base
         '#EC407A', '#AB47BC', '#42A5F5', '#66BB6A']   # bagging (mas intensos)
ROSA_SEQ = ['#FCE4EC', '#F8BBD0', '#F48FB1', '#EC407A', '#C2185B', '#880E4F']

print("Estilo visual configurado.")

## 2. Inspección de la carpeta de datosReviso la carpeta `exercise_1_data`. Según el enunciado tengo 13 datasets, cada uno con un `.train` y un `.test`. Compruebo.

In [ ]:
data_folder = './exercise_1_data'
archivos = sorted(os.listdir(data_folder))

print("Total de archivos en la carpeta: " + str(len(archivos)))
print()

nombres_datasets = sorted(set(
    f.replace('.train', '').replace('.test', '') for f in archivos
    if f.endswith('.train') or f.endswith('.test')
))

print("Datasets encontrados (" + str(len(nombres_datasets)) + "):")
for i, nombre in enumerate(nombres_datasets, 1):
    tiene_train = nombre + '.train' in archivos
    tiene_test = nombre + '.test' in archivos
    estado = "completo" if (tiene_train and tiene_test) else "INCOMPLETO"
    print("  " + str(i) + ". " + nombre.ljust(20) + " [" + estado + "]")

## 3. Carga de un archivo individualAntes de montar funciones sofisticadas, empiezo por lo básico: abrir un archivo y ver qué pinta tiene. Son archivos de texto con columnas separadas por espacios. La última columna es la clase (1.0 o -1.0) y el resto son features.

In [ ]:
ejemplo = np.loadtxt(os.path.join(data_folder, 'banana.train'))
print("Forma del array: " + str(ejemplo.shape))
print("Primeras 5 filas:")
print(ejemplo[:5])
print()
print("Ultima columna (clases): " + str(np.unique(ejemplo[:, -1])))

## 4. Función `load_datafile`Creo una función que reciba la ruta de un archivo, lo cargue con `np.loadtxt` y devuelva por separado las features (X) y las clases (y). Me aseguro de que y sea un array 1D con `.ravel()`, que si no luego sklearn se queja más que yo un lunes por la mañana.

In [ ]:
def load_datafile(filepath):
    """
    Carga un archivo .train o .test y devuelve X (features) e y (clase).
    La ultima columna es la variable objetivo.
    """
    data = np.loadtxt(filepath)
    X = data[:, :-1]
    y = data[:, -1].ravel()
    return X, y

X_train_banana, y_train_banana = load_datafile("./exercise_1_data/banana.train")
print("banana.train -> X: " + str(X_train_banana.shape) + ", y: " + str(y_train_banana.shape))
print("Clases: " + str(np.unique(y_train_banana)))

## 5. Función `load_dataset`Subo un nivel: una función que dado el nombre base del dataset carga automáticamente el .train y el .test y lo organiza en un diccionario.

In [ ]:
def load_dataset(base_path):
    """
    Devuelve {"train": (X_train, y_train), "test": (X_test, y_test)}
    """
    X_train, y_train = load_datafile(base_path + ".train")
    X_test, y_test = load_datafile(base_path + ".test")
    return {"train": (X_train, y_train), "test": (X_test, y_test)}

data_banana = load_dataset("./exercise_1_data/banana")
print("banana -> train: " + str(data_banana["train"][0].shape) + ", test: " + str(data_banana["test"][0].shape))

## 6. Función `load_datasets` (carga masiva)Recorro la carpeta, identifico todos los datasets y los cargo de golpe en un diccionario de diccionarios. Copiar y pegar 13 veces lo mismo no es trabajar, es sufrir.

In [ ]:
def load_datasets(folder='./exercise_1_data'):
    """Carga todos los datasets. Devuelve dict indexado por nombre."""
    archivos = os.listdir(folder)
    nombres = sorted(set(
        f.replace('.train', '').replace('.test', '')
        for f in archivos if f.endswith('.train')
    ))
    datasets = {}
    for nombre in nombres:
        base_path = os.path.join(folder, nombre)
        if os.path.exists(base_path + '.train') and os.path.exists(base_path + '.test'):
            datasets[nombre] = load_dataset(base_path)
    return datasets

datasets = load_datasets()

print("Datasets cargados: " + str(len(datasets)))
print()
for nombre, data in datasets.items():
    X_tr = data['train'][0]
    X_te = data['test'][0]
    info = nombre.ljust(20) + ' -> train: ' + str(X_tr.shape[0]).rjust(5)
    info += ' x ' + str(X_tr.shape[1]).rjust(2) + ' | test: ' + str(X_te.shape[0]).rjust(5)
    print(info)

Guardo el objeto `datasets` en un pickle para reutilizarlo en otros ejercicios.

In [ ]:
import pickle
with open('datasets.pkl', 'wb') as f:
    pickle.dump(datasets, f)
print("datasets.pkl guardado.")

## 7. Selección de clasificadores baseElijo 4 algoritmos de familias bien distintas entre sí (nada de ensembles tipo Random Forest):- **Logistic Regression (lr):** modelo lineal, rápido y estable. El clásico que nunca falla.- **K-Nearest Neighbors (knn):** basado en distancias, no paramétrico. Se adapta bien a fronteras no lineales.- **Decision Tree (dt):** basado en reglas, inestable por naturaleza. Justo lo que necesito para que el bagging brille.- **SVM con kernel RBF (svm):** potente para fronteras complejas, aunque más lento.La diversidad importa: si todos los modelos fuesen parecidos, combinarlos no aportaría nada. Necesito que se equivoquen en cosas distintas para que el ensemble compense los errores.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

base_learners = [
    ("lr", LogisticRegression(max_iter=1000, random_state=42)),
    ("knn", KNeighborsClassifier(n_neighbors=5)),
    ("dt", DecisionTreeClassifier(random_state=42)),
    ("svm", SVC(kernel="rbf", random_state=42))
]

print("Base learners seleccionados:")
for nombre, modelo in base_learners:
    print("  " + nombre + ": " + modelo.__class__.__name__)

## 8. Entrenamiento y evaluación de los modelos baseEntreno cada modelo en cada dataset y mido el accuracy en test. Doble bucle y au, aquí en Murcia somos eficientes.

In [ ]:
base_scores = pd.DataFrame(index=sorted(datasets.keys()))

for nombre_modelo, modelo in base_learners:
    accs = []
    for ds_name in sorted(datasets.keys()):
        X_train, y_train = datasets[ds_name]['train']
        X_test, y_test = datasets[ds_name]['test']
        clf = clone(modelo)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        accs.append(accuracy_score(y_test, y_pred))
    base_scores[nombre_modelo] = accs

base_scores = base_scores.round(4)
print("Accuracies de los modelos base:")
display(base_scores)
print()
print("Media por modelo:")
print(base_scores.mean().round(4).sort_values(ascending=False))

## 9. Visualización de resultados basePinto las accuracies para ver de un vistazo qué modelo gana en cada dataset.

In [ ]:
fig, ax = plt.subplots(figsize=(14, 7))
base_scores.plot(kind='bar', ax=ax, color=ROSA4, edgecolor='white', width=0.75)
ax.set_title('Accuracy de los modelos base por dataset', fontsize=14, fontweight='bold', color='#880E4F')
ax.set_xlabel('')
ax.set_ylabel('Accuracy', fontsize=11)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.legend(title='Modelo', bbox_to_anchor=(1.02, 1), loc='upper left')
ax.set_ylim(0.4, 1.05)
plt.tight_layout()
plt.show()

print("Ganador por dataset:")
for ds in base_scores.index:
    winner = base_scores.loc[ds].idxmax()
    val = base_scores.loc[ds].max()
    print("  " + ds.ljust(20) + " -> " + winner + " (" + str(round(val, 4)) + ")")

Observaciones rápidas: SVM gana en media, el Decision Tree destaca en image y thyroid (fronteras complejas), y LR aguanta bien en muchos datasets porque al fin y al cabo es un modelo robusto y generalista. No hay un ganador universal, que es justo lo que dice el Teorema de No Free Lunch.

## 10. Ensembles con BaggingBagging (Bootstrap Aggregating): entreno 50 copias del mismo modelo, cada una con una muestra bootstrap distinta del training set, y combino por votación mayoritaria.La gracia del bagging es que si el modelo base es inestable (como el Decision Tree, que cambia mucho con pequeñas variaciones en los datos), cada copia se equivocará en cosas distintas y al promediar se reduce la varianza del error.Uso `n_jobs=-1` para aprovechar todos los cores. Si el portátil echa humo es que está trabajando, como mi abuela con el caldero de arroz.

In [ ]:
from sklearn.ensemble import BaggingClassifier

bagging_scores = pd.DataFrame(index=sorted(datasets.keys()))

for nombre_modelo, modelo in base_learners:
    accs = []
    for ds_name in sorted(datasets.keys()):
        X_train, y_train = datasets[ds_name]['train']
        X_test, y_test = datasets[ds_name]['test']
        bag = BaggingClassifier(
            estimator=clone(modelo),
            n_estimators=50,
            random_state=42,
            n_jobs=-1
        )
        bag.fit(X_train, y_train)
        y_pred = bag.predict(X_test)
        accs.append(accuracy_score(y_test, y_pred))
    bagging_scores[nombre_modelo + '_bag'] = accs

bagging_scores = bagging_scores.round(4)
print("Accuracies con Bagging:")
display(bagging_scores)

## 11. Comparativa: base vs bagging

In [ ]:
all_scores = pd.concat([base_scores, bagging_scores], axis=1)

fig, ax = plt.subplots(figsize=(16, 8))
all_scores.plot(kind='bar', ax=ax, color=ROSA8, edgecolor='white', width=0.85)
ax.set_title('Base vs Bagging: accuracy por dataset', fontsize=14, fontweight='bold', color='#880E4F')
ax.set_ylabel('Accuracy')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.legend(title='Modelo', bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=9)
ax.set_ylim(0.4, 1.05)
plt.tight_layout()
plt.show()

In [ ]:
# Tabla de mejora: bagging vs base
print("Comparativa de medias (base vs bagging):")
print("=" * 55)
for nombre_modelo, _ in base_learners:
    media_base = base_scores[nombre_modelo].mean()
    media_bag = bagging_scores[nombre_modelo + '_bag'].mean()
    diff = media_bag - media_base
    flecha = "sube" if diff > 0 else "baja" if diff < 0 else "igual"
    linea = '  ' + nombre_modelo.ljust(5) + ': '
    linea += str(round(media_base, 4)) + ' -> ' + str(round(media_bag, 4))
    linea += ' (' + ('+' if diff >= 0 else '') + str(round(diff, 4)) + ', ' + flecha + ')'
    print(linea)

El resultado es coherente con la teoría:- El **Decision Tree** es el que más mejora con bagging (+0.036 de media). Es un modelo inestable por naturaleza: pequeños cambios en los datos producen árboles muy diferentes. Eso es exactamente lo que bagging necesita para generar diversidad real.- **KNN** y **SVM** mejoran ligeramente. Son modelos más estables, así que las 50 copias son bastante parecidas entre sí. La diversidad es baja y el ensemble apenas aporta.- **Logistic Regression** prácticamente no cambia. Es el modelo más estable de todos: le das datos distintos y sigue trazando la misma recta. 50 copias de LR son 50 veces lo mismo.Conclusión parcial: bagging funciona bien cuando el modelo base es inestable. Con modelos estables es tirar recursos de computación.

## 12. Ensembles con Boosting (AdaBoost)Boosting entrena modelos secuencialmente: cada nuevo modelo se centra en los ejemplos que los anteriores han fallado, reponderándolos con más peso. Es como un profe particular que insiste en los temas que peor llevas.Uso `AdaBoostClassifier` con `algorithm='SAMME'` porque el SAMME.R por defecto no funciona con todos los estimadores.**¿Por qué no hay `n_jobs` en AdaBoost?** Porque el boosting es secuencial por diseño. Cada modelo necesita los resultados del anterior para saber qué ejemplos reponderar. No se puede paralelizar. Esto es la diferencia fundamental con bagging, donde los modelos son independientes y sí se entrenan en paralelo.Importante: no todos los clasificadores son compatibles con AdaBoost. Necesita que el estimador base soporte `sample_weight` en `.fit()`. KNN no lo soporta, así que lo capturo con un try/except y sigo.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

# Para boosting, uso un DT poco profundo (weak learner) que es lo ideal
boost_learners = [
    ("lr", LogisticRegression(max_iter=1000, random_state=42)),
    ("knn", KNeighborsClassifier(n_neighbors=5)),
    ("dt", DecisionTreeClassifier(max_depth=3, random_state=42)),
    ("svm", SVC(random_state=42)),
]

boosting_scores = pd.DataFrame(index=sorted(datasets.keys()))
modelos_excluidos = []

for nombre_modelo, modelo in boost_learners:
    accs = []
    compatible = True
    for ds_name in sorted(datasets.keys()):
        X_train, y_train = datasets[ds_name]['train']
        X_test, y_test = datasets[ds_name]['test']
        try:
            boost = AdaBoostClassifier(
                estimator=clone(modelo),
                n_estimators=50,
                algorithm='SAMME',
                random_state=42
            )
            boost.fit(X_train, y_train)
            y_pred = boost.predict(X_test)
            accs.append(accuracy_score(y_test, y_pred))
        except Exception as e:
            compatible = False
            print("  " + nombre_modelo + " no compatible con AdaBoost: " + str(type(e).__name__))
            modelos_excluidos.append(nombre_modelo)
            break

    if compatible:
        boosting_scores[nombre_modelo + '_boo'] = accs
    else:
        print("  -> Salto " + nombre_modelo + " para boosting.")

boosting_scores = boosting_scores.round(4)
print()
print("Accuracies con Boosting:")
display(boosting_scores)

### Análisis del comportamiento del boostingAquí hay cosas muy interesantes que comentar:**¿Por qué KNN no es compatible con AdaBoost?** Porque AdaBoost necesita que el estimador base acepte pesos de muestra (`sample_weight`) en su método `.fit()` para poder reponderar los ejemplos difíciles. KNN no implementa esa funcionalidad: clasifica por cercanía y punto, no admite que le digan "oye, este ejemplo pesa más que aquel".**¿Por qué SVM se hunde en boosting?** Esto es lo más llamativo de los resultados. SVM con kernel RBF es un modelo fuerte y estable: ya por sí solo consigue un accuracy alto. El problema es que AdaBoost está diseñado para combinar *weak learners* (modelos simples que son solo un poco mejores que el azar). Cuando le metes un modelo fuerte como SVM pasan dos cosas: (1) los pesos de rebalanceo se vuelven muy extremos porque el modelo "casi acierta todo" y las pocas muestras que falla se sobrepesan de forma desproporcionada, y (2) el algoritmo SAMME tiene problemas numéricos cuando el error del base learner es muy bajo o muy alto. El resultado es que en vez de mejorar, destroza al modelo original. En banana pasa de 0.87 a 0.44, que es peor que tirar una moneda.**El Decision Tree en cambio mejora notablemente.** Un DT con max_depth=3 es un weak learner perfecto para boosting: acierta más que el azar pero comete suficientes errores como para que el rebalanceo tenga sentido. Es la combinación ideal.

In [ ]:
all_scores = pd.concat([all_scores, boosting_scores], axis=1)

# Genero paleta pastel para todos los modelos
import colorsys
from matplotlib.colors import to_hex

def paleta_rosa_n(n):
    colores = []
    for i in range(n):
        # Voy de rosa a lila a azul pastel
        hue = 0.85 + (i / n) * 0.45  # rango de tonos rosa-lila-azul
        if hue > 1.0:
            hue -= 1.0
        rgb = colorsys.hls_to_rgb(hue, 0.75, 0.5)
        colores.append(to_hex(rgb))
    return colores

fig, ax = plt.subplots(figsize=(18, 8))
paleta = paleta_rosa_n(len(all_scores.columns))
all_scores.plot(kind='bar', ax=ax, color=paleta, edgecolor='white', width=0.88)
ax.set_title('Base vs Bagging vs Boosting', fontsize=14, fontweight='bold', color='#880E4F')
ax.set_ylabel('Accuracy')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.legend(title='Modelo', bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=8)
ax.set_ylim(0.4, 1.05)
plt.tight_layout()
plt.show()

print("Medias globales hasta ahora:")
print(all_scores.mean().sort_values(ascending=False).round(4))

## 13. Ensembles heterogéneos: Voting y StackingHasta ahora he combinado copias del mismo modelo (ensembles homogéneos). Ahora mezclo modelos distintos:- **Voting:** cada modelo vota una clase y gana la mayoría. Es una democracia de algoritmos.- **Stacking:** cada modelo hace su predicción y un meta-modelo (en mi caso una Logistic Regression) aprende a combinar esas predicciones de la mejor forma. Es como tener un jefe que sabe a quién hacer caso según el tema.`mixing_scores` tiene solo 2 columnas porque voting y stacking son técnicas que combinan todos los base learners a la vez en un único ensemble. No hay una versión "por modelo" como en bagging o boosting.

In [ ]:
from sklearn.ensemble import VotingClassifier, StackingClassifier

mixing_scores = pd.DataFrame(index=sorted(datasets.keys()))

# --- VOTING ---
accs_vote = []
for ds_name in sorted(datasets.keys()):
    X_train, y_train = datasets[ds_name]['train']
    X_test, y_test = datasets[ds_name]['test']
    voting = VotingClassifier(
        estimators=[(n, clone(m)) for n, m in base_learners],
        voting='hard', n_jobs=-1
    )
    voting.fit(X_train, y_train)
    y_pred = voting.predict(X_test)
    accs_vote.append(accuracy_score(y_test, y_pred))
mixing_scores['vote'] = accs_vote

# --- STACKING ---
accs_stack = []
for ds_name in sorted(datasets.keys()):
    X_train, y_train = datasets[ds_name]['train']
    X_test, y_test = datasets[ds_name]['test']
    stacking = StackingClassifier(
        estimators=[(n, clone(m)) for n, m in base_learners],
        final_estimator=LogisticRegression(max_iter=1000, random_state=42),
        cv=5, n_jobs=-1
    )
    stacking.fit(X_train, y_train)
    y_pred = stacking.predict(X_test)
    accs_stack.append(accuracy_score(y_test, y_pred))
mixing_scores['stack'] = accs_stack

mixing_scores = mixing_scores.round(4)
print("Accuracies con Voting y Stacking:")
display(mixing_scores)

## 14. Análisis final: ranking, ganadores y conclusiones

In [ ]:
all_scores = pd.concat([base_scores, bagging_scores, boosting_scores, mixing_scores], axis=1)

print("GANADOR POR DATASET:")
print("=" * 55)
winners = all_scores.idxmax(axis=1)
for ds in winners.index:
    winner = winners[ds]
    val = all_scores.loc[ds, winner]
    print("  " + ds.ljust(20) + " -> " + winner.ljust(12) + " (" + str(round(val, 4)) + ")")

print()
ranking = all_scores.mean().sort_values(ascending=False).round(4)
print("RANKING GLOBAL (media de accuracy en los 13 datasets):")
print("=" * 55)
for i, (modelo, media) in enumerate(ranking.items(), 1):
    print("  " + str(i).rjust(2) + ". " + modelo.ljust(14) + " " + str(media))

In [ ]:
# Grafico de victorias
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

wins = winners.value_counts()
wins.plot(kind='bar', ax=axes[0], color=ROSA_SEQ[:len(wins)], edgecolor='white')
axes[0].set_title('Victorias por modelo', fontsize=13, fontweight='bold', color='#880E4F')
axes[0].set_ylabel('Datasets ganados')
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45, ha='right')
for i, v in enumerate(wins.values):
    axes[0].text(i, v + 0.1, str(v), ha='center', fontweight='bold')

# Ranking de medias
colores_rank = []
mediana_rank = ranking.median()
for v in ranking.values:
    if v >= ranking.values[0] - 0.005:
        colores_rank.append('#EC407A')
    elif v >= mediana_rank:
        colores_rank.append('#F48FB1')
    else:
        colores_rank.append('#F8BBD0')
ranking.plot(kind='barh', ax=axes[1], color=colores_rank, edgecolor='white')
axes[1].set_title('Ranking por accuracy media', fontsize=13, fontweight='bold', color='#880E4F')
axes[1].set_xlabel('Accuracy media')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

In [ ]:
# Heatmap completo
from matplotlib.colors import LinearSegmentedColormap

rosa_cmap = LinearSegmentedColormap.from_list('rosa',
    ['#FFFFFF', '#FCE4EC', '#F8BBD0', '#F48FB1', '#EC407A', '#C2185B'])

fig, ax = plt.subplots(figsize=(16, 9))
sns.heatmap(all_scores, annot=True, fmt='.3f', cmap=rosa_cmap, ax=ax,
            linewidths=0.5, linecolor='white', vmin=0.5, vmax=1.0)
ax.set_title('Accuracy de todos los modelos en todos los datasets',
             fontsize=13, fontweight='bold', color='#880E4F')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
plt.tight_layout()
plt.show()

### Análisis de diversidad entre modelos basePara entender por qué los ensembles heterogéneos (voting, stacking) funcionan bien, voy a medir cuánto "discrepan" los modelos base entre sí. Si todos predicen lo mismo, combinarlos no aporta nada. Si discrepan mucho en sus errores, el ensemble puede compensarlos.Calculo el porcentaje de desacuerdo entre cada par de modelos: en cuántos ejemplos de test uno acierta y el otro falla (o viceversa).

In [ ]:
# Mido la discrepancia (disagreement) entre pares de modelos
# en cada dataset, luego promedio
from itertools import combinations

pares = list(combinations([n for n, _ in base_learners], 2))
disagreement = pd.DataFrame(index=sorted(datasets.keys()), columns=[p[0]+'_vs_'+p[1] for p in pares])

for ds_name in sorted(datasets.keys()):
    X_train, y_train = datasets[ds_name]['train']
    X_test, y_test = datasets[ds_name]['test']
    preds = {}
    for nombre_modelo, modelo in base_learners:
        clf = clone(modelo)
        clf.fit(X_train, y_train)
        preds[nombre_modelo] = clf.predict(X_test)
    for (m1, m2) in pares:
        disag = (preds[m1] != preds[m2]).mean()
        disagreement.loc[ds_name, m1 + '_vs_' + m2] = round(disag, 4)

disagreement = disagreement.astype(float)
print("Porcentaje de desacuerdo entre pares de modelos (media en todos los datasets):")
print("=" * 55)
for col in disagreement.columns:
    media = disagreement[col].mean()
    print("  " + col.ljust(12) + ": " + str(round(media * 100, 1)) + "%")

print()
print("Cuanto mayor sea el desacuerdo, mas potencial tiene el ensemble para mejorar.")
print("Si dos modelos siempre predicen lo mismo, combinarlos no aporta nada nuevo.")

Los modelos que más discrepan entre sí son los que más se benefician de ser combinados. Esto explica por qué el stacking (que aprende a combinar las predicciones de los 4 modelos) consigue el mejor ranking global: aprovecha tanto la precisión individual como la diversidad entre modelos.

### Conclusiones1. **No hay un modelo universal.** Depende del dataset. SVM gana en unos, DT en otros, LR en otros. Es el Teorema de No Free Lunch en estado puro.2. **Bagging mejora a los inestables.** El Decision Tree sube un 3.6% de media con bagging. LR y SVM apenas se inmutan porque son modelos estables: 50 copias de lo mismo siguen siendo lo mismo.3. **Boosting necesita weak learners.** Con DT poco profundo funciona genial, pero con SVM se va al garete porque es un modelo demasiado fuerte. AdaBoost no sabe qué hacer con un estimador que ya acierta casi todo. KNN directamente no es compatible porque no soporta sample_weight.4. **Voting y stacking son consistentes.** Rara vez ganan en un dataset concreto, pero rara vez son los peores. El stacking es el ganador del ranking global porque tiene un meta-modelo que aprende a combinar las fortalezas de cada clasificador.5. **La diversidad es la clave.** He medido el desacuerdo entre modelos y los pares que más discrepan son los que más se benefician de ser combinados. Sin diversidad, no hay ensemble que valga.

## 15. Ejercicio avanzado: mi propio super-ensembleVoy a construir un "ensemble de ensembles": un stacking donde los estimadores base ya son ensembles:- **Componente 1:** BaggingClassifier de árboles (captura la inestabilidad del DT)- **Componente 2:** AdaBoostClassifier de stumps (se centra en los ejemplos difíciles)- **Componente 3:** VotingClassifier heterogéneo (diversidad de familias)Un meta-modelo de Logistic Regression decide cómo ponderarlos. Es el típico movimiento murciano: si el plato ya está bueno, le echamos más cosas. A veces funciona, a veces te pasas con la ñora.

In [ ]:
advanced_scores = pd.DataFrame(index=sorted(datasets.keys()))
accs_super = []

for ds_name in sorted(datasets.keys()):
    X_train, y_train = datasets[ds_name]['train']
    X_test, y_test = datasets[ds_name]['test']

    bag_tree = BaggingClassifier(
        estimator=DecisionTreeClassifier(random_state=42),
        n_estimators=50, random_state=42, n_jobs=-1
    )

    ada_stump = AdaBoostClassifier(
        estimator=DecisionTreeClassifier(max_depth=2, random_state=42),
        n_estimators=50, algorithm='SAMME', random_state=42
    )

    voter = VotingClassifier(
        estimators=[
            ("lr", LogisticRegression(max_iter=1000, random_state=42)),
            ("knn", KNeighborsClassifier(n_neighbors=5)),
            ("svm", SVC(random_state=42))
        ],
        voting='hard', n_jobs=-1
    )

    super_ensemble = StackingClassifier(
        estimators=[
            ("bag_tree", bag_tree),
            ("ada_stump", ada_stump),
            ("voter", voter)
        ],
        final_estimator=LogisticRegression(max_iter=1000, random_state=42),
        cv=5, n_jobs=-1
    )

    super_ensemble.fit(X_train, y_train)
    y_pred = super_ensemble.predict(X_test)
    accs_super.append(accuracy_score(y_test, y_pred))

advanced_scores['super_ensemble'] = accs_super
advanced_scores = advanced_scores.round(4)

print("Accuracies del super-ensemble:")
display(advanced_scores)
print()
media_s = str(round(advanced_scores['super_ensemble'].mean(), 4))
print("Media global: " + media_s)

In [ ]:
all_final = pd.concat([all_scores, advanced_scores], axis=1)
ranking_final = all_final.mean().sort_values(ascending=False).round(4)

print("RANKING FINAL DEFINITIVO:")
print("=" * 55)
for i, (modelo, media) in enumerate(ranking_final.items(), 1):
    marca = "  <-- mi criatura" if modelo == "super_ensemble" else ""
    print("  " + str(i).rjust(2) + ". " + modelo.ljust(18) + " " + str(media) + marca)

print()
wins_s = int((all_final.idxmax(axis=1) == 'super_ensemble').sum())
total = len(all_final)
print("El super-ensemble gana en " + str(wins_s) + " de " + str(total) + " datasets.")

In [ ]:
# Heatmap definitivo
fig, ax = plt.subplots(figsize=(18, 9))
sns.heatmap(all_final, annot=True, fmt='.3f', cmap=rosa_cmap, ax=ax,
            linewidths=0.5, linecolor='white', vmin=0.5, vmax=1.0)
ax.set_title('Mapa de calor definitivo: todos los modelos y ensembles',
             fontsize=14, fontweight='bold', color='#880E4F')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
plt.tight_layout()
plt.show()

print("13 datasets, un monton de modelos, y la conclusion de siempre:")
print("juntos somos mas fuertes que solos. Pero hay que saber juntarse bien.")